In [1]:
from datetime import timedelta, date
import fetch
from keplergl import KeplerGl
import pandas as pd
import sqlite3

In [2]:
def getMonth(curr_month):
    month = fetch.get_weather(usecols='time,lat,lon,avg(`WindSpd_ms-1`)', datetime=date(2015,curr_month, 1),groupBy='lat,lon')
    conn = sqlite3.connect('../database/database.db')
    if curr_month == 12:
        monthlen = (date(2016, 1, 1) - date(2015, curr_month, 1)).days - 1
    else:
        monthlen = (date(2015, curr_month + 1, 1) - date(2015, curr_month, 1)).days -1

    for currDay in (date(2015,curr_month, 2) + timedelta(n) for n in range(monthlen)):
        df = fetch.get_weather(usecols='time, lat, lon, avg(`WindSpd_ms-1`)', datetime=currDay, groupBy='lat, lon', conn=conn)
        if not df.empty:
            month = pd.concat([month, df], ignore_index=True)
            
    month['time'] = month['time'].apply(lambda x: pd.Timestamp(x[:10] + ' ' + x[11:]))

    conn.close()

    return month

In [3]:
months = [None]

for i in range(1, 13):
    months.append(getMonth(i))

In [4]:
%run config.py

In [5]:
monthlyMap = KeplerGl(height=600,data={'month': months[5]}, config=dailyAvgWindSpeed)

monthlyMap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['month'], 'id': 'nmadvrb4', …

In [8]:
day = fetch.get_weather(usecols='time,lat,lon,`WindSpd_ms-1`', datetime=date(2015,10, 19))
day['time'] = day['time'].apply(lambda x: pd.Timestamp(x[:10] + ' ' + x[11:]))

dailyMap = KeplerGl(height=600,data={'day': day}, config=dailyWindSpeed)
dailyMap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['day'], 'id': 'xmowh9a9w', '…